In [15]:
import os
import json
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image

In [5]:
class ConditionalZiziDataset(Dataset):
    def __init__(self, input_dir, image_size):
        self.img_dir = input_dir + "train_img"
        self.pose_dir = input_dir + "train_openpose"
        self.img_files = os.listdir(self.img_dir)
        self.img_files.sort()
        self.image_size = image_size
        self.preprocess = transforms.Compose(
            [
                transforms.Resize((image_size, image_size)),
                transforms.ToTensor(),
                transforms.Normalize([0.5], [0.5])
            ]
        )

    def __len__(self):
        return len(self.img_files)

    def _get_img(self, idx):
        return tensor.zeros(1)
        # img_dir = os.path.join(self.img_dir, self.img_files[idx])
        # with Image.open(img_dir) as img:
        #     # img.thumbnail((self.image_size, self.image_size))
        #     return self.preprocess(img.convert("RGB"))

    def _get_json(self, idx):
        input_split = os.path.splitext(self.img_files[idx])
        pose_json_name = input_split[0] + "_keypoints.json"
        json_dir = os.path.join(self.pose_dir, pose_json_name)
        with open(json_dir, 'r') as f:
            data = json.load(f)
        return torch.tensor(data['people'][0]['pose_keypoints_2d'])
    
    def __getitem__(self, idx):
        if isinstance(idx, slice):
            # Get the start, stop, and step from the slice
            return [self[ii] for ii in range(*idx.indices(len(self)))]

        input_img = self._get_img(idx)
        json_data = self._get_json(idx)

        return {
            "images": input_img,
            "poses": json_data
        }


In [20]:
x = ConditionalZiziDataset("data/pink-cape-me/", 128)

In [26]:
xlen = [len(y["poses"]) for y in x]

In [29]:
len([z for z in xlen if z != 75])

0